<a href="https://colab.research.google.com/github/alex-jk/SWB-GVCEH/blob/main/models/relevance_model/Relevance_Model_Reddit_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install setfit
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=118942c7b5f1ed785516f1558fee37ead0527b47dd3b21dc4161d18574f58e66
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import pandas as pd
from setfit import SetFitModel
from tqdm.notebook import tqdm

##### Import reddit datasets

In [4]:
file_path = '/content/drive/My Drive/SWB-GVCEH/VictoriaBC_data_updated.csv'
VictoriaBC_data_nodups = pd.read_csv(file_path)
VictoriaBC_data_nodups = VictoriaBC_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nVictoria BC data ----------------")
print(VictoriaBC_data_nodups.shape)
print(VictoriaBC_data_nodups.columns)
print(VictoriaBC_data_nodups.head())

Sooke_data_nodups = pd.read_csv('/content/drive/My Drive/SWB-GVCEH/Sooke_data.csv')
Sooke_data_nodups = Sooke_data_nodups.drop_duplicates().reset_index(drop=True)

print("\nSooke data ----------------")
print(Sooke_data_nodups.shape)
print(Sooke_data_nodups.columns)
print(Sooke_data_nodups.head())


Victoria BC data ----------------
(8998, 6)
Index(['Subreddit', 'Title', 'Text', 'Submission ID', 'User ID',
       'Search Term'],
      dtype='object')
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                              Free clothing places?   

                                                Text Submission ID  \
0  I've started my own organization to expose the...        zen3ao   
1                                                NaN        5yah6z   
2  Went to the Christmas tree farm today and thou...       17yjdiw   
3                                                NaN        7h0e58   
4  Hey so long story short I moved here from Sask...       169ywlj   

          User 

In [5]:
file_url = 'https://drive.google.com/uc?id=1ANE3_UkNi2UGyQpHr8Ujz1QwBNKxmcTl'

# Reading the CSV file into a DataFrame
additional_reddit_df = pd.read_csv(file_url)
additional_reddit_df = additional_reddit_df.drop_duplicates().reset_index(drop=True)
additional_reddit_df.rename(columns={'Body': 'Text'}, inplace=True)

print("\nAdditional reddit data ----------------")
print(additional_reddit_df.shape)
print(additional_reddit_df.columns)
print(additional_reddit_df.head())


Additional reddit data ----------------
(2057, 4)
Index(['Subreddit', 'Title', 'Text', 'Comments'], dtype='object')
               Subreddit                                              Title  \
0  OakBayBritishColumbia                            Oak Bay high right now.   
1  OakBayBritishColumbia  Food share for Ukrainian refugees on Vancouver...   
2  OakBayBritishColumbia     Lost Budgie - Cadboro Bay (Willows Elementary)   
3  OakBayBritishColumbia  What are 3 things I must see when visiting Oak...   
4  OakBayBritishColumbia  Yes, this is a leaf blower rant, but hear me o...   

                                                Text  \
0  Anyone know what the hell is going on at Oak B...   
1                                                NaN   
2  My friends blue budgie flew out of the house l...   
3                                            Thanks!   
4  I have a neighbour who uses his leaf blower to...   

                                            Comments  
0  ['https://www

##### Combine all data into one df

In [6]:
# Combining the DataFrames
select_cols = ['Subreddit', 'Title', 'Text']
combined_df = pd.concat([VictoriaBC_data_nodups[select_cols], Sooke_data_nodups[select_cols], additional_reddit_df[select_cols]], ignore_index=True)

# Resetting the index
combined_df.reset_index(drop=True, inplace=True)
print("\Combined data ----------------")
print(combined_df.shape)
print(combined_df.columns)
print(combined_df.head())

\Combined data ----------------
(11160, 3)
Index(['Subreddit', 'Title', 'Text'], dtype='object')
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                              Free clothing places?   

                                                Text  
0  I've started my own organization to expose the...  
1                                                NaN  
2  Went to the Christmas tree farm today and thou...  
3                                                NaN  
4  Hey so long story short I moved here from Sask...  


##### Load twitter data

In [7]:
# List of file names
file_names = [
    'GVCEH-tweets-combined_2023-02-08.csv',
    'GVCEH-tweets-combined_2023-01-30.csv',
    'GVCEH-tweets-combined_2023-01-21.csv',
    'GVCEH-tweets-combined_2023-01-12.csv'
]

# Base URL for raw files in the GitHub repository
base_url = 'https://raw.githubusercontent.com/alex-jk/SWB-GVCEH/main/data/processed/twitter/github_actions/'

# Initialize a list to collect the DataFrames
dfs = []

for file_name in file_names:
    # Construct the full URL for the current file
    file_url = base_url + file_name
    # Read the CSV file
    current_df = pd.read_csv(file_url)
    # Append the DataFrame to the list
    dfs.append(current_df)

# Concatenate all DataFrames in the list
tweets_combined_df = pd.concat(dfs, ignore_index=True)
# Remove duplicates
tweets_combined_df = tweets_combined_df.drop_duplicates().reset_index(drop=True)

# Displaying the first few rows of the DataFrame
print(tweets_combined_df.shape)
print(tweets_combined_df.columns)
print(tweets_combined_df.head())

(5435, 17)
Index(['Unnamed: 0', 'text', 'scrape_time', 'tweet_id', 'created_at',
       'reply_count', 'quote_count', 'like_count', 'retweet_count',
       'geo_full_name', 'geo_id', 'username', 'num_followers',
       'search_keywords', 'search_neighbourhood', 'sentiment', 'score'],
      dtype='object')
   Unnamed: 0                                               text  \
0           0  RT pressjournal: Colonsay islanders and people...   
1           1  Colonsay islanders and people who have left th...   
2           7  @ArianeBurgessHI Serviced plots for 25k are ex...   
3           9  RT @VicBuilders: "25-unit townhome development...   
4          27  @OurNewHomecoach @laughatthemoon2 There is so ...   

                  scrape_time             tweet_id                 created_at  \
0  2023-02-07 03:20:43.040309  1622564995115503616  2023-02-06 11:56:55+00:00   
1  2023-02-07 03:20:43.040317  1622550741599625221  2023-02-06 11:00:16+00:00   
2  2023-02-07 03:20:51.207543  1622549961

##### Check twitter data

In [8]:
ind = 8
print(tweets_combined_df['search_neighbourhood'][ind])
print(tweets_combined_df['search_keywords'][ind])
print(tweets_combined_df['text'][ind])

burnside-gorge OR fairfield-gonzales OR hollywood park OR north park OR pauquachin OR salt spring island OR stadacona park OR victoria
(burnside-gorge OR fairfield-gonzales OR hollywood park OR north park OR pauquachin OR salt spring island OR stadacona park OR victoria) (anawin companion society OR safer victoria OR vtag OR mental health recovery partners, south island OR vancouver island mental health society OR greater victoria acting together OR the mustard seed OR yyj tenants union OR pacifica housing OR solid outreach OR housing OR camper) lang:en -is:retweet
Can you trust @Dave_Eby?One week he says he's not buying in Burnside GorgeThen they announce he (as Housing Minister) "bought a hotel (Capital City Centee) adjacent to Downtown"Adjacent = Burnside Gorge@Adam_Stirling, what do you think?Can Eby be trusted? https://t.co/6yYCcXAvvC @mattdellok @Stephen_Andrew @BC_Housing @Dave_Eby @VictoriaDRA "It would be easier to buy in Burnside Gorge" Minister Eby https://t.co/bSP0LdgPwJ


In [9]:
# Load the pretrained SetFit model
model = SetFitModel.from_pretrained("sheilaflood/gvceh-setfit-rel-model2")

# Example text data
texts = ["Example text relevant to homelessness in Victoria.", "Irrelevant text about other topics."]

# Model makes predictions
predictions = model(texts)
print(predictions)

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/7.48k [00:00<?, ?B/s]

tensor([1, 0])


In [10]:
ind = 7
model(tweets_combined_df['text'][ind])

tensor(1)

##### Check that tweets are considered relevant by the model
- all tweets were found to be relevant by the model

In [ ]:
# Define a function to make predictions
def get_prediction(text):
    prediction = model([text])[0]
    return prediction

# Apply the model to each row in the 'text' column with a progress bar
tqdm.pandas()  # Enable tqdm for pandas
tweets_combined_df['relevant'] = tweets_combined_df['text'].progress_apply(get_prediction)

  0%|          | 0/5435 [00:00<?, ?it/s]

In [ ]:
def extract_integer_from_tensor(tensor_val):
    return tensor_val.item()

# Apply this function to the entire column
# tweets_combined_df['relevant'] = tweets_combined_df['relevant'].apply(extract_integer_from_tensor)

# Print value counts for the 'relevant' column
print(tweets_combined_df['relevant'].value_counts())

1    5435
Name: relevant, dtype: int64


##### Perform K-Means clustering of twitter data